In [1]:
import subprocess
from tqdm import tqdm

In [2]:
def checked_call(command):
    call = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    if call.returncode != 0:
        raise Exception("Failed call: " + str(command) + "\n" + call.stdout.decode())
    return call

## Runs all tests in Tests folder and saves results
Only works on windows, powershell 7.0 or later, python 3 with standard scientific packages (esp. pandas and tqdm) are required.

Docker and the mp-fsm-eval image must be installed. Docker must also be running.

pwsh (powershell7 executable) and python (python3 executable) and docker must be accessable from PATH.

### Get all .json files

In [3]:
# get all jsons
command = [
    "pwsh",
    "-Command",
    "Get-ChildItem", 
    ".\\Tests",
    "-Filter *.json",
    "-Recurse | % { $_.FullName }"
]
call = checked_call(command)

In [4]:
jsons = call.stdout.decode().split()
jsons

['E:\\GitRepMain\\tesi-mpc-2020\\Tests\\example-computation.json',
 'E:\\GitRepMain\\tesi-mpc-2020\\Tests\\increasing-party-fixed-others.json',
 'E:\\GitRepMain\\tesi-mpc-2020\\Tests\\increasing-states-fixed-others.json',
 'E:\\GitRepMain\\tesi-mpc-2020\\Tests\\increasing-string-fixed-others.json',
 'E:\\GitRepMain\\tesi-mpc-2020\\Tests\\increasing-symbols-fixed-others.json',
 'E:\\GitRepMain\\tesi-mpc-2020\\Tests\\spdz-semi-comparison.json']

### Generate .ps1 scripts

In [5]:
# generate all scripts
scripts = []
for json in tqdm(jsons):
    # generate script destination
    script_dest = json.split("\\")
    script_dest[-2] = "Scripts"
    script_dest[-1] = script_dest[-1][:-4] + "ps1"
    script_dest = "\\".join(script_dest)
    
    # generate script
    command = [
        "pwsh",
        "-Command",
        "python",
        ".\\generate_test_script.py",
        json, 
        script_dest
    ]
    call = checked_call(command)
    scripts.append(script_dest)
scripts

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:11<00:00,  1.96s/it]


['E:\\GitRepMain\\tesi-mpc-2020\\Scripts\\example-computation.ps1',
 'E:\\GitRepMain\\tesi-mpc-2020\\Scripts\\increasing-party-fixed-others.ps1',
 'E:\\GitRepMain\\tesi-mpc-2020\\Scripts\\increasing-states-fixed-others.ps1',
 'E:\\GitRepMain\\tesi-mpc-2020\\Scripts\\increasing-string-fixed-others.ps1',
 'E:\\GitRepMain\\tesi-mpc-2020\\Scripts\\increasing-symbols-fixed-others.ps1',
 'E:\\GitRepMain\\tesi-mpc-2020\\Scripts\\spdz-semi-comparison.ps1']

### Run all .ps1 scripts (Docker must be open)

In [6]:
raw_results = []
for script in tqdm(scripts):
    # generate script
    command = [
        "pwsh",
        "-Command",
        script
    ]
    call = checked_call(command)
    raw_results.append(call.stdout.decode())

100%|███████████████████████████████████████████████████████████████████████████████████| 6/6 [14:27<00:00, 144.50s/it]


### Save Results

In [7]:
raw_files = []
csv_files = []
for script, raw_result in tqdm(zip(scripts, raw_results), total=len(scripts)):
    # generate raw file name
    raw_dest = script.split("\\")
    raw_dest[-2] = "Results\\Raw"
    raw_dest[-1] = raw_dest[-1][:-3] + "txt"
    raw_dest = "\\".join(raw_dest)
    
    # generate raw file
    with open(raw_dest, "w+", newline="\n") as fd:
        fd.write(raw_result)
        
    raw_files.append(raw_dest)
        
    csv_dest = raw_dest.split("\\")
    csv_dest[-2] = "Parsed"
    csv_dest[-1] = csv_dest[-1][:-3] + "csv"
    csv_dest = "\\".join(csv_dest)
    
    # parse files (if available)
    command = [
        "pwsh",
        "-Command",
        "python",
        ".\\parse_test_results.py",
        raw_dest,
        csv_dest
    ]
    call = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    if call.returncode != 0:
        continue
    csv_files.append(csv_dest)

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:17<00:00,  2.93s/it]


In [8]:
raw_files

['E:\\GitRepMain\\tesi-mpc-2020\\Results\\Raw\\example-computation.txt',
 'E:\\GitRepMain\\tesi-mpc-2020\\Results\\Raw\\increasing-party-fixed-others.txt',
 'E:\\GitRepMain\\tesi-mpc-2020\\Results\\Raw\\increasing-states-fixed-others.txt',
 'E:\\GitRepMain\\tesi-mpc-2020\\Results\\Raw\\increasing-string-fixed-others.txt',
 'E:\\GitRepMain\\tesi-mpc-2020\\Results\\Raw\\increasing-symbols-fixed-others.txt',
 'E:\\GitRepMain\\tesi-mpc-2020\\Results\\Raw\\spdz-semi-comparison.txt']

In [9]:
csv_files

['E:\\GitRepMain\\tesi-mpc-2020\\Results\\Parsed\\increasing-party-fixed-others.csv',
 'E:\\GitRepMain\\tesi-mpc-2020\\Results\\Parsed\\increasing-states-fixed-others.csv',
 'E:\\GitRepMain\\tesi-mpc-2020\\Results\\Parsed\\increasing-string-fixed-others.csv',
 'E:\\GitRepMain\\tesi-mpc-2020\\Results\\Parsed\\increasing-symbols-fixed-others.csv',
 'E:\\GitRepMain\\tesi-mpc-2020\\Results\\Parsed\\spdz-semi-comparison.csv']

## Extra

### Script Usage

In [10]:
command = [
    "pwsh",
    "-Command",
    "python",
    ".\\generate_test_script.py",
    "--help"
]
call = checked_call(command)
print(call.stdout.decode())

usage: generate_test_script.py [-h] [-I IMAGE] [-N NETWORK]
                               [-C CONTAINER_PREFIX] [-S SHELL]
                               json name

Generate a Test Suite for MP-FSM evaluation given json file

positional arguments:
  json                  Path to json file describing the test
  name                  Name of generated Script

optional arguments:
  -h, --help            show this help message and exit
  -I IMAGE, --image IMAGE
                        Name of the image (default: mp-fsm-eval)
  -N NETWORK, --network NETWORK
                        Name of the network (default: mp-fsm-eval-internal-
                        lan)
  -C CONTAINER_PREFIX, --container-prefix CONTAINER_PREFIX
                        Prefix for Container names (default: mp-fsm-eval)
  -S SHELL, --shell SHELL
                        Specifies shell (default: powershell7). Currently
                        supported shells are: bash (.sh script), powershell7
                        (

In [11]:
command = [
    "pwsh",
    "-Command",
    "python",
    ".\\parse_test_results.py",
    "--help"
]
call = checked_call(command)
print(call.stdout.decode())

usage: parse_test_results.py [-h] file csv

Parses results generated by any 'generate_test_script.py' script. Note that
'DEBUG MODE' is not supported

positional arguments:
  file        Path to file to be parsed
  csv         Name of generated .csv

optional arguments:
  -h, --help  show this help message and exit

